In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import importlib
import re

import pandas as pd
import sqlalchemy as sa

import pudl

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_settings

# Sales by Rate Schedule

### Request 2:
* **p304 L8(b)** TOTAL RESIDENTIAL - MWh Sold (difficult b/c rate schedules change)
* **p304 L8(c)** TOTAL RESIDENTIAL - Revenue (difficult b/c rate schedules change)
* **p304 L8(d)** TOTAL RESIDENTIAL - Avg Number of Customers (difficult b/c rate schedules change)
* **p304 L24(b)** TOTAL SM/LG C&I - MWh Sold (difficult b/c rate schedules change)
* **p304 L24(c)** TOTAL SM/LG C&I - Revenue (difficult b/c rate schedules change)
* **p304 L24(d)** TOTAL SM/LG C&I - Avg. Number of Customers (difficult b/c rate schedules change)
* **p304 L43(b)** TOTAL (MWh Sold?) (doable b/c it's a specific designated row)
* **p304 L43(c)** TOTAL (Revenue?) (doable b/c it's a specific designated row)
* **p304 L43(d)** TOTAL (Avg. Number of Customers?) (doable b/c it's a specific designated row)

Note that due to lack of any kind of standardization or categorization in the rate schedule nomenclature, and the variety that exists between all the different states and across years, it is extremely difficult to extract anything useful from this table. Even the `total` line is kind of a mess. However, the same information can be pulled from the `f1_elctrc_oper_rev` table (p300) below.

In [ ]:
sales_by_sched = (
    pd.read_sql("f1_sales_by_sched", ferc1_engine).
    pipe(pudl.transform.ferc1.unpack_table,
         table_name="f1_sales_by_sched",
         data_cols=[
             "mwh_sold",
             "revenue",
             "avg_num_cstmr"
         ],
         data_rows=[
             "total"
         ])
    .droplevel(1, axis="columns")
    .dropna(how="all")
    .query("report_prd==12")
    .reset_index()
    .set_index(["respondent_id", "report_year", "spplmnt_num"])
    .drop("report_prd", axis="columns")
    .rename(columns={
        "avg_num_cstmr": "avg_customers",
    })
    .astype({"avg_customers": "Int64"})
)

# O&M Expenses

### Request 2:
* **p320 L5(b)** (Steam) Fuel (501)
* **p320 L25(b)** (Nuclear) Fuel (518)
* **p320 L63(b)** (Other) Fuel (547)
* **p321 L76(b)** Purchased Power (555)

In [ ]:
elec_oandm = (
    pd.read_sql("f1_elc_op_mnt_expn", ferc1_engine)
    .pipe(
        pudl.transform.ferc1.unpack_table,
        table_name="f1_elc_op_mnt_expn",
        data_cols=[
            "crnt_yr_amt"
        ],
        data_rows=[
            "production_steam_ops_acct501_fuel",
            "production_nuclear_ops_acct518_fuel",
            "production_other_ops_acct547_fuel",
            "production_supply_acct555_purchased_power",
        ])
    .droplevel(0, axis="columns")
    .dropna(how="all")
    .query("report_prd==12")
    .reset_index()
    .set_index(["respondent_id", "report_year"])
    .drop(["report_prd", "spplmnt_num"], axis="columns")
    .rename_axis(columns=None)
)

# Operating Revenues

### Request 2:
* **p300 L10(b,d,f)** TOTAL Sales to Ultimate Consumers (revenues, MWh, avg customers)
* **p300 L11(b,d,f)** Sales for Resale (447) (revenues, MWh, avg customers)
* **p300 L12(b,d,f)** TOTAL Sales of Electricity (revenues, MWh, avg customers)

### Additional Data in lieu of `f1_sales_by_sched`
* **p300 L2(b,d,f)** Residential Sales (440) (revenues, MWh, avg customers)
* **p300 L4(b,d,f)** Commercial and Industrial Sales, Small (442) (revenues, MWh, avg customers)
* **p300 L5(b,d,f)** Commercial and Industrial Sales, Large (442) (revenues, MWh, avg customers)


In [ ]:
sales_hierarchical = (
    pd.read_sql("f1_elctrc_oper_rev", ferc1_engine)
    .pipe(
        pudl.transform.ferc1.unpack_table,
        table_name="f1_elctrc_oper_rev",
        data_cols=[
            "rev_amt_crnt_yr",
            "mwh_sold_crnt_yr",
            "avg_cstmr_crntyr"
        ],
        data_rows=[
            "sales_acct440_residential",
            "sales_acct442_commercial_industrial_small",
            "sales_acct442_commercial_industrial_large",
            "sales_acct447_for_resale",
            "sales_ultimate_consumers_total",
            "sales_of_electricity_total",
            "sales_revenues_net_total",
        ])
    .dropna(how="all")
    .query("report_prd==12")
    .reset_index()
    .set_index(["respondent_id", "report_year"])
    .drop([("report_prd", ""), ("spplmnt_num", "")], axis="columns")
    .rename_axis(columns=[None, None])
    .assign(avg_cstmr_crntyr=lambda x: x.loc[:, "avg_cstmr_crntyr"].astype("Int64"))
)

sales_revenue = (
    sales_hierarchical.loc[:, "rev_amt_crnt_yr"]
    .add_suffix("_revenue")
)
sales_mwh = (
    sales_hierarchical.loc[:, "mwh_sold_crnt_yr"]
    .add_suffix("_mwh")
)
sales_customers = (
    sales_hierarchical.loc[:, "avg_cstmr_crntyr"]
    .add_suffix("_customers")
)

elec_sales = pd.concat([sales_revenue, sales_mwh, sales_customers], axis="columns")
elec_sales = elec_sales.loc[:,elec_sales.columns.sort_values()]

# Income Statements

## Request 1
* **p115 L2(g+h)** Electric Operating Revenues Current+Prior Year
* **p115 L?(g+h)** Electric Operating Expenses Current+Prior Year
* **p115 L26(g+h)** Electric Net Util Oper Inc Current Year Current+Prior Year

## Request 2
* **p114 L6(g)** Depreciation Expense (403)
* **p114 L7(g)** Depreciation Expense for Asset Retirement Costs (403.1)
* **p114 L8(g)** Amort. & Depl. of Utility Plant (404-405)
* **p114 L9(g)** Amort. of Uitlity Plant Acq. Adj. (406)
* **p114 L10(g)** Amort. of Property Losses, Urecov Plant and Reg. Study Costs (407)
* **p114 L11(g)** Amort. of Conversion Expenses (407)
* **p114 L14(g)** Taxes other than Income taxes (408.1)
* **p114 L15(g)** Income Taxes - Federal (409.1)
* **p114 L16(g)** Other (409.1)
* **p114 L17(g)** Provision for Deferred Income Taxes (410.1)
* **p114 L18(g)** (Less) Provision for Deferred Tax Credits (411.1)
* **p114 L19(g)** Investment Tax Credit Adj. - Net (411.4) 

In [ ]:
elec_income = (
    pd.read_sql("f1_income_stmnt", ferc1_engine)
    .pipe(
        pudl.transform.ferc1.unpack_table,
        table_name="f1_income_stmnt",
        data_cols = [
            "cy_elctrc_total",
        ],
        data_rows = [
            "operating_revenues_acct400",
            "depreciation_expenses_acct403",
            "depreciation_expenses_asset_retirement_acct403_1",
            "amortization_depletion_utility_plant_acct404_405",
            "amortization_utility_plant_acquired_acct406",
            "amortized_conversion_expenses_acct407",
            "amortized_losses_acct407",
            "non_income_tax_acct408_1",
            "federal_income_tax_acct409_1",
            "other_acct409_1",
            "deferred_income_tax_acct410_1",
            "deferred_income_tax_credit_acct411_1",
            "investment_tax_credit_acct411_4",
            "utility_operating_expenses_total",
            "net_utility_operating_income",
        ])
    .dropna(how="all")
    .query("report_prd==12")
    .droplevel(0, axis="columns")
    .reset_index()
    .set_index(["respondent_id", "report_year"])
    .drop(["report_prd", "spplmnt_num"], axis="columns")
    .rename_axis(columns=None)
)

# Depreciation

### Request 1:
* **P336 L12(f)** Electric Depreciation Expense

In [ ]:
elec_depreciation = (
    pd.read_sql("f1_dacs_epda", ferc1_engine)
    .pipe(
        pudl.transform.ferc1.unpack_table,
        table_name="f1_dacs_epda",
        data_cols=['total'],
        data_rows=["total_electric_plant"]
    )
    .dropna(how="all")
    .query("report_prd==12")
    .droplevel(0, axis="columns")
    .reset_index()
    .set_index(["respondent_id", "report_year"])
    .drop(["report_prd", "spplmnt_num"], axis="columns")
    .rename_axis(columns=None)
)

# Plant in Service

### Request 1:
* **P206 L104(b)** TOTAL Electric Plant in Service Bal Beginning of Year
* **P206 L104(g)** TOTAL Electric Plant in Service Bal End of Year

In [ ]:
elec_plant_in_service = (
    pd.read_sql("f1_plant_in_srvce", ferc1_engine)
    .pipe(
        pudl.transform.ferc1.unpack_table,
        table_name="f1_plant_in_srvce",
        data_cols=["begin_yr_bal", "yr_end_bal"],
        data_rows=["electric_plant_in_service_total"]
    )
    .dropna(how="all")
    .query("report_prd==12")
    .droplevel(1, axis="columns")
    .reset_index()
    .set_index(["respondent_id", "report_year"])
    .drop(["report_prd", "spplmnt_num"], axis="columns")
    .rename(columns={
        "begin_yr_bal": "starting_balance",
        "yr_end_bal": "ending_balance",
    })
)

# EIA 861 Sales by Customer Class

## Request 2:

**Pull:**
  * Annual revenues (USD)
  * Annual sales (MWh)
  * Annual customers counts

**For each of:**
  * Residential customers
  * Commercial customers
  * Industrial customers
  * All Customers (even though are others like Transportation sales in there?)

In [ ]:
import pudl.extract.eia861
eia861_years = range(1999,2019)
eia861_extractor = pudl.extract.eia861.ExtractorExcel(
    dataset_name="eia861",
    years=eia861_years,
    pudl_settings=pudl_settings
)
eia861_dfs = eia861_extractor.create_dfs(years=eia861_years)
sales_eia861 = eia861_dfs["sales_eia861_states"]
cols = [
    "report_year",
    "utility_id_eia",
    "state",
    
    "residential_revenues",
    "residential_sales_mwh",
    "residential_customers",
    
    "commercial_revenues",
    "commercial_sales_mwh",
    "commercial_customers",
    
    "industrial_revenues",
    "industrial_sales_mwh",
    "industrial_customers",
    
    "transportation_revenues",
    "transportation_sales_mwh",
    "transportation_customers",
    
    "other_revenues",
    "other_sales_mwh",
    "other_customers",
    
    "total_revenues",
    "total_sales_mwh",
    "total_customers",
]
sales_eia861 = (
    sales_eia861.loc[:,cols].reset_index(drop=True)
    .query("utility_id_eia not in (88888, 99999)")
    .assign(report_year=lambda x: pd.to_numeric(x.report_year, errors="coerce"))
    .dropna(subset=["report_year", "utility_id_eia"])
    .astype({"report_year": int, "utility_id_eia": int}, errors="ignore")
)

rev_cols = sales_eia861.filter(regex=".*_revenues$").columns
for col in rev_cols:
    sales_eia861.loc[:,col] = 1000.0 * pd.to_numeric(sales_eia861[col], errors="coerce")

cust_cols = sales_eia861.filter(regex=".*_customers$").columns
for col in cust_cols:
    sales_eia861.loc[:,col] = pd.to_numeric(sales_eia861[col], errors="coerce").astype("Int64")

mwh_cols = sales_eia861.filter(regex=".*_sales_mwh$").columns
for col in mwh_cols:
    sales_eia861.loc[:,col] = pd.to_numeric(sales_eia861[col], errors="coerce")

#new_df = pd.DataFrame()
#for customer_class in ["residential", "commercial", "industrial", "transportation", "other", "total"]:
#    tmp_df = (
#        sales_eia861.set_index(["utility_id_eia", "report_year", "state"])
#        .filter(regex=f"^{customer_class}_.*")
#        .assign(customer_class=customer_class)
#        .rename(columns=lambda x: re.sub(f"^{customer_class}_", "", x))
#    )
#    new_df = new_df.append(tmp_df)

#sales_eia861 = (
#    new_df.reset_index()
#    .assign(
#        revenues=lambda x: 1000.0 * pd.to_numeric(x.revenues, errors="coerce"),
#        customers=lambda x: pd.to_numeric(x.customers, errors="coerce"),
#        sales_mwh=lambda x: pd.to_numeric(x.sales_mwh, errors="coerce")
#    )
#    .astype({"customers": "Int64"})
#    .set_index(["utility_id_eia", "state", "report_year", "customer_class"])
#    .reset_index()
#)

In [ ]:
ferc1_dfs = {
    "elec_oandm_ferc1": elec_oandm,
    "elec_sales_ferc1": elec_sales,
    "elec_income_ferc1": elec_income,
    "elec_depreciation_ferc1": elec_depreciation,
    "elec_plant_in_service_ferc1": elec_plant_in_service,
}

# Check Data Quality

## Spot check dataframes

In [ ]:
elec_plant_in_service.columns

In [ ]:
elec_depreciation.columns

In [ ]:
print(elec_plant_in_service.count())

In [ ]:
df = pd.merge(elec_plant_in_service, elec_depreciation, left_index=True, right_index=True)

In [ ]:
sns.scatterplot(x="total_electric_plant", y="ending_balance", data=df)

In [ ]:
elec_income.columns

In [ ]:
elec_income.sample(10)

In [ ]:
df = elec_income.reset_index()
df["net_income_calculated"] = df.operating_revenues_acct400 - df.utility_operating_expenses_total
income_totals = [
    "operating_revenues_acct400",
    "utility_operating_expenses_total",
    "net_utility_operating_income",
    "net_income_calculated",
]
for col in income_totals:
    sns.lineplot(x="report_year", y=col, data=df, estimator="sum", label=col)
plt.ylabel("USD")
plt.legend()
plt.show();

In [ ]:
sns.scatterplot(x="net_utility_operating_income", y="net_income_calculated", data=df)

In [ ]:
df = elec_sales.reset_index()
mwh_cols = df.filter(regex=".*acct.*mwh$").columns
df = pudl.transform.ferc1.oob_to_nan(df, mwh_cols, ub=1e9)
cust_cols = df.filter(regex=".*acct.*customers$").columns
df.loc[:,cust_cols] = df.loc[:,cust_cols].astype(float)
rev_cols = df.filter(regex=".*acct.*revenue$").columns

In [ ]:
mwh_cols

In [ ]:
for var in mwh_cols:
    sns.lineplot(x="report_year", y=var, data=df, estimator="mean", label=var)
    plt.ylabel("Electricity Sold [MWh]")
    plt.xlabel(None)
    plt.legend()
    plt.show()


In [ ]:
for var in cust_cols:
    sns.lineplot(x="report_year", y=var, data=df, estimator="mean", label=var)
    plt.ylabel("Number of Customers")
    plt.xlabel(None)
    plt.legend()
    plt.show()


In [ ]:
df.query("respondent_id==151")

In [ ]:
sns.lineplot(x="report_year", y="sales_acct447_for_resale_customers", data=df, units="respondent_id", estimator=None)

In [ ]:
for var in rev_cols:
    sns.lineplot(x="report_year", y=var, data=df, estimator="mean", label=var)
plt.ylabel("Electricity Revenues [USD]")
plt.xlabel(None)
plt.legend()
plt.show()

In [ ]:
mwh_cols

In [ ]:
sns.scatterplot(x="sales_acct440_residential_mwh", y="sales_acct440_residential_revenue", data=df, alpha=0.1, label="Residential")
plt.show()
sns.scatterplot(x="sales_acct442_commercial_industrial_small_mwh", y="sales_acct442_commercial_industrial_small_revenue", data=df, alpha=0.1, label="Small C&I")
plt.show()
sns.scatterplot(x="sales_acct442_commercial_industrial_large_mwh", y="sales_acct442_commercial_industrial_large_revenue", data=df, alpha=0.1, label="Large C&I")
plt.show()

In [ ]:
df = elec_sales.reset_index()
for var in df.filter(regex=".*acct.*customers$").columns:
    sns.lineplot(x="report_year", y=var, data=df, estimator="sum", label=var)

plt.ylabel("Customers")
plt.xlabel(None)
plt.legend()

In [ ]:
df = elec_oandm.reset_index()
for var in df.filter(regex="^production_.*").columns:
    sns.lineplot(x="report_year", y=var, data=df, estimator="sum", label=var.split('_')[1])
plt.ylabel("[USD]")
plt.xlabel(None)
plt.title("Total Fuel / Purchased Power Costs")
plt.legend()
plt.show()

In [ ]:
sns.lineplot(x="report_year", y="production_steam_ops_acct501_fuel", data=df, units="respondent_id", estimator=None, alpha=0.1)
plt.show()
sns.lineplot(x="report_year", y="production_nuclear_ops_acct518_fuel", data=df, units="respondent_id", estimator=None, alpha=0.1)
plt.show()
sns.lineplot(x="report_year", y="production_other_ops_acct547_fuel", data=df, units="respondent_id", estimator=None, alpha=0.1)
plt.show()
sns.lineplot(x="report_year", y="production_supply_acct555_purchased_power", data=df, units="respondent_id", estimator=None, alpha=0.1)
plt.show()

# Prepare Data for Output

## Select requested utilities
* Use Binz' target list to select a subset of the tables/columns.
* Add missing & unmapped `utility_id_ferc1` values to the (FERC) target list:
    - **`eia 22500`** : `ferc1 191, 276` (Westar Energy)
    - **`eia 13780`** : `ferc1 121` (Northern States Power Company - WI)
    - **`eia 13809, 13902`** : `ferc1 122` (Northwestern Public Service Co)
* Merge in additional utility name/ID fields for readability.

In [ ]:
# Grab the EIA/FERC Utility IDs & Names:
utilities_eia = pd.read_sql("utilities_eia", pudl_engine)
utilities_ferc1 = pd.read_sql("utilities_ferc1", pudl_engine)

# Get Binz' list of utilities based on EIA IDs:
rbinz_eia_utils = pd.read_csv("rbinz_instructions.csv", index_col="utility_id_eia")

# Infer FERC 1 Utility IDs for Binz' targets:
utility_id_eia_targets = (
    rbinz_eia_utils
    .merge(utilities_eia, how="left", on="utility_id_eia", suffixes=("_rbinz", "_pudl"))
    .astype({"utility_id_pudl": 'Int64'})
    .dropna(subset=["utility_id_pudl"])
    .merge(utilities_ferc1, how="left", on="utility_id_pudl")
    .astype({"utility_id_ferc1": 'Int64'})
    .set_index("utility_id_eia")
    .dropna(subset=["utility_id_ferc1"])
)

# Add in a few FERC1 IDs that were missing:
utility_id_ferc1_targets = set(utility_id_eia_targets.utility_id_ferc1).union({121, 122, 191, 276})

In [ ]:
binz_out = {}
for k in ferc1_dfs.keys():
    binz_out[k]= (
        ferc1_dfs[k].reset_index()
        .rename(columns={"respondent_id": "utility_id_ferc1"})
        .query("utility_id_ferc1 in @utility_id_ferc1_targets")
        .merge(utilities_ferc1, on="utility_id_ferc1")
        .merge(utilities_eia[["utility_id_pudl", "utility_id_eia"]], on="utility_id_pudl")
        .set_index(["utility_id_ferc1", "report_year", "utility_name_ferc1", "utility_id_eia", "utility_id_pudl"])
        .reset_index()
    )
    print(f"{len(binz_out[k])} records in {k}")


In [ ]:
all_target_eia_ids = set(utility_id_eia_targets.reset_index().utility_id_eia).union({22500, 13780, 13809, 13902})
binz_out["elec_sales_eia861"] = (
    sales_eia861
    .merge(utilities_eia, on="utility_id_eia", how="left")
    .query("utility_id_eia in @all_target_eia_ids")
    .merge(utilities_ferc1, on="utility_id_pudl", how="left")
    .set_index(["utility_id_eia", "report_year", "utility_name_eia", "utility_id_pudl", "utility_id_ferc1", "utility_name_ferc1", "state"])
    .reset_index()
    .astype({"utility_id_pudl": "Int64",
             "utility_id_ferc1": "Int64"})
)

In [ ]:
for df in binz_out:
    print(f"Writing {df}.csv")
    binz_out[df].to_csv(f"{df}.csv", index=False)